## Семинар 13: "Обучение с подкреплением 1"

ФИО: __Федотова Алёна Дмитриевна__

###  FrozenLake


<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>



In [2]:
import gym
import numpy as np 

#create a single game instance
env = gym.make("FrozenLake-v0")

#start new game
env.reset();

In [3]:
# display the game state
env.render()


SFFF
FHFH
FFFH
HFFG


### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [7]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

initial observation code: 0
printing observation:

SFFF
FHFH
FFFH
HFFG
observations: Discrete(16) n= 16
actions: Discrete(4) n= 4


In [11]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
new observation code: 1
reward: 0.0
is game over?: False
printing new state:
  (Right)
SFFF
FHFH
FFFH
HFFG


In [12]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

### Попробуйте походить по замерзшему озеру, не упав в дырку. 
* Каждый шаг вы с вероятностью __0.5__ будете двигаться в выбранном направлении и с вероятностью __0.5__ в случайном.
* Если упадете, используйте __env.reset()__ чтобы перезапустить __env__

In [53]:
env.reset()

0

In [57]:
new_obs, reward, is_done, _ = env.step(action_to_i['right'])
print(is_done)
env.render()

True
  (Right)
SFFF
FHFH
FFFH
HFFG


## Задание: 
Следуя шаблонам функций реализуйте алгоритм Policy iteration.
Протестируйте его с помощью функции __evaluate_policy__.
Попробуйте разные значение для gamma и сравните результаты.

In [131]:
def run_episode(env, policy, gamma = 1.0, render = False):
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        # total_reward += (gamma ** step_idx * reward)
        total_reward += reward # иначе получается бессмысленно
        step_idx += 1
        if done:
            break
    return total_reward

In [59]:
def evaluate_policy(env, policy, gamma = 1.0,  n = 100):
    scores = [
            run_episode(env, policy, gamma = gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

In [60]:
env.env.P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [85]:
# Используя фиксированную value_function и жадную стратегию получите policy

def extract_policy(v, gamma = 1.0):
    policy = np.zeros(env.env.nS)
    q = np.zeros((env.env.nS, env.action_space.n))
    for s in env.env.P.keys():
        for a in env.env.P[s].keys():
            for p, s_, r, _ in env.env.P[s][a]:
                q[s, a] += gamma * p * v[s_] + r * p
        policy[s] = np.argmax(q[s, :])
    return policy

In [114]:
# Используя фиксированную policy с помощью сэмплирования получите оценку для value_function с точностью 1e-10


def compute_policy_v(env, policy, gamma=1.0):
    v_prev = np.ones(env.env.nS)
    v_new = np.zeros(env.env.nS)
    while np.linalg.norm(v_new - v_prev) > 1e-10:
        v_prev = v_new
        v_new = np.zeros(env.env.nS)
        for s in env.env.P.keys():
            for p, s_, r, _ in env.env.P[s][policy[s]]:
                v_new[s] += gamma * p * v_prev[s_] + r * p
    return v_new

In [87]:
def policy_iteration(env, gamma=1.0, max_iterations=50):
    policy = np.random.choice(np.arange(env.env.nA), env.env.nS)
    gamma = 1.0
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env, policy, gamma=1.0)
        new_policy = extract_policy(old_policy_v, gamma = 1.0)
        if (np.all(policy == new_policy)):
            break

        policy = new_policy
    return policy

In [132]:
gamma = 1.0
env = gym.make("FrozenLake-v0")
optimal_policy = policy_iteration(env, gamma=gamma, max_iterations=10000)
policy_score = evaluate_policy(env, optimal_policy, gamma, n=10000)

print(optimal_policy)
print(policy_score)

[ 0.  3.  3.  3.  0.  0.  0.  0.  3.  1.  0.  0.  0.  2.  1.  0.]
0.7382


In [133]:
gamma = 0.5
env = gym.make("FrozenLake-v0")
optimal_policy = policy_iteration(env, gamma=gamma, max_iterations=10000)
policy_score = evaluate_policy(env, optimal_policy, gamma, n=10000)

print(optimal_policy)
print(policy_score)

[ 0.  3.  3.  3.  0.  0.  0.  0.  3.  1.  0.  0.  0.  2.  1.  0.]
0.7393


In [134]:
gamma = 0.2
env = gym.make("FrozenLake-v0")
optimal_policy = policy_iteration(env, gamma=gamma, max_iterations=10000)
policy_score = evaluate_policy(env, optimal_policy, gamma, n=10000)

print(optimal_policy)
print(policy_score)

[ 0.  3.  3.  3.  0.  0.  0.  0.  3.  1.  0.  0.  0.  2.  1.  0.]
0.7456


In [135]:
gamma = 0.1
env = gym.make("FrozenLake-v0")
optimal_policy = policy_iteration(env, gamma=gamma, max_iterations=10000)
policy_score = evaluate_policy(env, optimal_policy, gamma, n=10000)

print(optimal_policy)
print(policy_score)

[ 0.  3.  3.  3.  0.  0.  0.  0.  3.  1.  0.  0.  0.  2.  1.  0.]
0.7376


In [136]:
gamma = 0.0
env = gym.make("FrozenLake-v0")
optimal_policy = policy_iteration(env, gamma=gamma, max_iterations=10000)
policy_score = evaluate_policy(env, optimal_policy, gamma, n=10000)

print(optimal_policy)
print(policy_score)

[ 0.  3.  3.  3.  0.  0.  0.  0.  3.  1.  0.  0.  0.  2.  1.  0.]
0.7401


Получилось, что от $\gamma$ ничего не зависит. Можно заметить, что каждый раз мы находим одну и ту же стратегию. Судя по всему, она оптимальна вообще среди всех стратегий, так как игра не очень сложная.